In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, GRU
from keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier
from tqdm import tqdm

In [2]:
# ハイパーパラメータの設定
look_back = 30
test_size = 0.2
n_splits = 5
epochs = 100
batch_size = 32

In [3]:
# 株価データの読み込み
# df = pd.read_csv("stock_data.csv")
df = pd.read_pickle(r"C:\Users\trow1\Documents\my_test\powerX_strategy\data\df_rates_1H.pkl")

In [4]:
df = df[df['time'] >= "1999-1-1"]
df = df.rename(columns={
    "open": "Open",
    "high": "High", 
    "low": "Low",
    "close": "Close",
    "tick_volume": "Volume"
})


In [5]:
# 前処理
df.dropna(inplace=True)
df = df[["Open", "High", "Low", "Close", "Volume"]]
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df)


In [6]:
# 特徴量とターゲットの作成
X, y = [], []
for i in range(len(scaled_data) - look_back - 1):
    a = scaled_data[i:(i + look_back), :]
    X.append(a)
    y.append(1 if df["Close"].iloc[i + look_back + 1] > df["Close"].iloc[i + look_back] else 0)
X, y = np.array(X), np.array(y)

In [7]:
# 時系列を考慮したデータ分割
tscv = TimeSeriesSplit(n_splits=n_splits)

In [8]:
# LSTMモデルの構築
def create_lstm_model(units=50, dropout_rate=0.2, optimizer='adam'):
    model = Sequential()
    model.add(LSTM(units, input_shape=(look_back, 5), return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# GRUモデルの構築
def create_gru_model(units=50, dropout_rate=0.2, optimizer='adam'):
    model = Sequential()
    model.add(GRU(units, input_shape=(look_back, 5), return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(GRU(units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# ハイパーパラメータのグリッドサーチ
param_grid = {
    'optimizer': ['adam', 'rmsprop']
}
# Early Stoppingの設定
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [9]:
# LSTMモデルでグリッドサーチ
lstm_model = KerasClassifier(build_fn=create_lstm_model, epochs=epochs, batch_size=batch_size, verbose=0, dropout_rate=0.2, units=100)
lstm_grid_search = GridSearchCV(estimator=lstm_model, param_grid=param_grid, cv=tscv, n_jobs=-1)
lstm_grid_search.fit(X, y, callbacks=[early_stopping])


c:\Users\trow1\AppData\Local\Programs\Python\Python312\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\trow1\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=KerasClassifier(batch_size=32, build_fn=<function create_lstm_model at 0x00000252F53063E0>, dropout_rate=0.2, epochs=100, units=100, verbose=0),
             n_jobs=-1, param_grid={'optimizer': ['adam', 'rmsprop']})

In [10]:
# GRUモデルでグリッドサーチ
gru_model = KerasClassifier(build_fn=create_gru_model, epochs=epochs, batch_size=batch_size, verbose=0, dropout_rate=0.2, units=100)
gru_grid_search = GridSearchCV(estimator=gru_model, param_grid=param_grid, cv=tscv, n_jobs=-1)
gru_grid_search.fit(X, y, callbacks=[early_stopping])


c:\Users\trow1\AppData\Local\Programs\Python\Python312\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\trow1\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=KerasClassifier(batch_size=32, build_fn=<function create_gru_model at 0x00000252F5306160>, dropout_rate=0.2, epochs=100, units=100, verbose=0),
             n_jobs=-1, param_grid={'optimizer': ['adam', 'rmsprop']})

In [11]:
# 最適なモデルの選択
best_model = lstm_grid_search if lstm_grid_search.best_score_ > gru_grid_search.best_score_ else gru_grid_search

# テストデータでの評価
for train_index, test_index in tqdm(tscv.split(X), total=n_splits):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)
    
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print(f"Precision: {precision_score(y_test, y_pred):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred):.4f}")
    print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")

# 売買シグナルの生成
signal = best_model.predict(X[-look_back:])
print(f"Buy" if signal else "Sell")

  0%|          | 0/5 [00:00<?, ?it/s]c:\Users\trow1\AppData\Local\Programs\Python\Python312\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\trow1\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\trow1\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
 20%|██        | 1/5 [37:03<2:2

Confusion Matrix:
[[13721     0]
 [12777     0]]
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000


c:\Users\trow1\AppData\Local\Programs\Python\Python312\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\trow1\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\trow1\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
 40%|████      | 2/5 [1:55:15<3:03:45, 3675.32s/it]

Confusion Matrix:
[[13518     0]
 [12980     0]]
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000


 40%|████      | 2/5 [3:07:01<4:40:31, 5610.61s/it]


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.
